In [1]:
import pandas as pd
import numpy 

import gurobipy as gp
from IPython.display import Image

In [2]:
# configs

data_path = '..\data\dados_trajeto.csv'

m = 12000
rho = 1.2754
Cd = 0.36
Cr = 0.9
Af = 6.48
g = 9.81

## 1.1 Função Objetivo

O = (n<sub>bp</sub> * n<sub>bs</sub> * c<sub>b</sub>) * (c<sub>fc</sub> + w<sub>fc</sub>) + $\mu$<sub>b</sub> $\left(\int_{t1}^{t2} Pb \; dt\right)$ + $\mu$<sub>fc</sub> $\left(\int_{t1}^{t2} Pfc \; dt\right)$

Em que: 

* n<sub>bp</sub> = n° de baterias em paralelo
* n<sub>bs</sub> = n° de baterias em série
* c<sub>b</sub> = custo unitário da bateria 
* c<sub>fc</sub> = custo Kg de combustível
* w<sub>fc</sub> = peso do tanque da célula de combustível
* $\mu$<sub>b</sub> = preço da energia elétrica em RS/kWh 
* $\left(\int_{t1}^{t2} Pb \; dt\right)$ = Energia proveniente das baterias
* $\mu$<sub>fc</sub> = preço do combustível em RS/kWh 
* $\left(\int_{t1}^{t2} Pfc \; dt\right)$ = Energia proveniente da célula combustível 

## 1.2 Variáveis de decisão

* n<sub>bp</sub>
* n<sub>bs</sub>
* w<sub>fc</sub>

## 1.3 Parâmetros

* c<sub>b</sub>
* c<sub>fc</sub>
* $\mu$<sub>b</sub>
* $\mu$<sub>fc</sub>
* m<sub>b</sub> = massa de cada bateria
* Q = capacidade da bateria
* V<sub>b</sub> = Tensão nominal
* V<sub>max</sub> = Tensão máxima
* I<sub>max</sub> = Saída máxima
* P<sub>max</sub> = Potência máxima da bateria
* P<sub>m</sub> = Potência média da bateria
* R = Resistência interna da bateria
* u<sub>fc</sub> = energia interna do hidrogênio ou outro combustível escolhido
* t = tempo da Viagem
* P<sub>d</sub> = Potência Demandada

P<sub>b</sub> = (n<sub>bs</sub>*V<sub>max</sub> * n<sub>bp</sub>*P<sub>max</sub>/V<sub>max</sub>  -   (n<sub>bp</sub>*P<sub>max</sub>/V<sub>max</sub>)² * n<sub>bp</sub>/n<sub>bs</sub>*R)

P<sub>fc</sub> = w<sub>fc</sub> * u<sub>fc</sub>

## 2. Modelando a demanda de potência do veículo

**Força de arrasto (Fa)**

Fa(t) = 0,5.ρ.Cd.Af.(v(t))²

em que:
* ρ é a densidade do ar
* Cd é o coeficiente de arrasto para o ônibus
* Af é a área da superfície frontal do veículo
* v(t) é a velocidade do veículo em função do tempo


*(Para fins de facilitar os cálculos nesta primeira fase de implementação, será considerado uma inclinação de **0 graus** no trajeto)*

**Força de resistência ao rolamento (Fr)**

Fr(t) = m.g.Cr.cos(§(t))

em que:
* m é a massa do veículo
* g é a aceleração da gravidade (9,81 m/s²)
* Cr é o coeficiente de atrito de rolamento
* §(t) é a inclinação do trajeto em função do tempo

**Força gravitacional (Fg)**

Fg(t) = m.g.Cr.sin(§(t))
(Será nula uma vez que foi considerado um angulo de inclinação de 0 graus)

**Força de aceleração (Facc)**

Facc(t) = m.(dvc(t)/dt)


A Força de tração é a soma de todas as forças citadas acima:

Ftr(t) = Facc(t) + Fg(t) + Fr(t) + Fa(t)

Assim, a demanda de potência pode ser dada por:

Ptr(t) = Ftr(t).v(t)

#### Parâmetros:
    
Cd = 0,36 -> Modelo de ônibus com os cantos da face arredondados Hucho (1986) <br>
ρ = 1,2754 kg/m³ <br>
Fr = 0,9 -> [fonte](https://www.if.ufrgs.br/~lang/Textos/Pneus.pdf) <br>
Af = 2,4x,2,7 = 6,48m² -> [fonte](https://www.ehow.com.br/tamanho-padrao-onibus-urbano-fatos_71396/) <br>
m = 12000 kg -> peso média de um ônibus médio -> [fonte](https://www.diariodasleis.com.br/legislacao/federal/61150-aprova-as-especificacoes-tecnicas-de-veiculos-automotores-de-transporte-coletivo-de-passageiros-rodoviario-e-urbano-intermunicipal.html#:~:text=%C3%94nibus%20leve%3A%20aquele%20com%20peso,ou%20superior%20a%2014%2C0t.) <br>

Usando os dados do TG do vinicius apenas como base para o teste

In [3]:
df = pd.read_csv(data_path, sep = ';')

df['tempo_s'] = df['tempo_s'] - 10
df['distancia_m'] = df['distancia_m'] - 155.42

delta = df.diff()

df['velocidade'] = delta['distancia_m']/delta['tempo_s']

delta = df.diff()

df['acc'] = abs(delta['velocidade'])/delta['tempo_s']
df.fillna(0, inplace=True)

df['Fa'] = 0.5*rho*Cd*Af*((df['velocidade'])**2)
df['Fr'] = m*g*Cr
df['Facc'] = m*df['acc']
df['Ftr'] = df['Fa'] + df['Fr'] + df['Facc']
df['pot'] = (df['Ftr']*df['velocidade'])/1000

In [4]:
df

,tempo_s,distancia_m,velocidade,acc,Fa,Fr,Facc,Ftr,pot
0,0,0.00,0.00,0.000000e+00,0.000000,105948.0,0.000000e+00,105948.000000,0.000000
1,1,11.95,11.95,0.000000e+00,212.436792,105948.0,0.000000e+00,106160.436792,1268.617220
2,2,23.91,11.96,1.000000e-02,212.792483,105948.0,1.200000e+02,106280.792483,1271.118278
3,3,35.86,11.95,1.000000e-02,212.436792,105948.0,1.200000e+02,106280.436792,1270.051220
4,4,47.82,11.96,1.000000e-02,212.792483,105948.0,1.200000e+02,106280.792483,1271.118278
...,...,...,...,...,...,...,...,...,...
1094,1094,5946.14,11.87,1.000000e-02,209.601971,105948.0,1.200000e+02,106277.601971,1261.515135
1095,1095,5958.03,11.89,2.000000e-02,210.308891,105948.0,2.400000e+02,106398.308891,1265.075893
1096,1096,5969.92,11.89,9.094947e-13,210.308891,105948.0,1.091394e-08,106158.308891,1262.222293
1097,1097,5981.83,11.91,2.000000e-02,211.017001,105948.0,2.400000e+02,106399.017001,1267.212292


In [5]:
Pot_demandada = df['pot'].sum(); Pot_demandada

665641.6809982802

In [6]:
t = 1098

## 3. Modelando o problema 

In [7]:
# Definindo o problema
modelo = gp.Model('Otimizacao')

Set parameter Username
Academic license - for non-commercial use only - expires 2022-06-20


In [21]:
# Definindo as variáveis de decisão
Ns = modelo.addVar(vtype= gp.GRB.BINARY, name="Ns")
Np = 1 #modelo.addVar(vtype= gp.GRB.INTEGER)
Wfc = 2 #modelo.addVar(vtype= gp.GRB.INTEGER)

In [22]:
# Criando função objetivo
modelo.setMObjective(Ns*Np*1089.99 + 20*Wfc, sense=gp.GRB.MINIMIZE)

TypeError: setMObjective() takes at least 4 positional arguments (2 given)

In [17]:
# Criando função objetivo
modelo.setMObjective((Ns*Np*1089.99) +  
                     (20*Wfc) + 
                     0.92*((Ns*12)*(Np*15) - ((Np*15)**2)*Ns/Np*0.8)*1098 + 
                     0.92*(80*Wfc)*1098, 
                     sense=gp.GRB.MINIMIZE)

TypeError: setMObjective() takes at least 4 positional arguments (2 given)

Modificar o problema para uma abordagem mais simples levando em conta a modelagem do problema da mochila.
o mais rápido possível.